<a href="https://colab.research.google.com/github/linyu3294/cs6220-data-minning-hw/blob/main/cs6220_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


---
## **PROBLEM 1: DBSCAN on toy-neighborhood data**

You are to cluster, and visualize, a small dataset using DBSCAN epsilon = 7.5, MinPts = 3). 

You have been provided a file, dbscan.csv, that has the following columns for each point in the dataset:
*   cluster originally empty, provided for your convenience pt a unique id for each data point
*  x point x-coordinate
*  y point y-coordinate
*   num neighbors number of neighbors, according to the coordinates above neighbors the id’s of all neighbors within

As you can see, a tedious O(n^2) portion of the work has been done for you. 

Your job is to execute, point-by-point, the DBSCAN algorithm, logging your work.



---

## **PROBLEM 2: DBSCAN on toy raw data**

Three toy 2D datasets are provided (or they can be obtained easily with scikit learn) circles; blobs, and moons. Run your own implementaion of DBSCAN on these, in two phases.

---
### **Section A**
Run your own implementaion of DBSCAN on CIRCLES, in two phases.


---
### **Section B**
Run your own implementaion of DBSCAN on BLOBS, in two phases.


---
### **Section C**
Run your own implementaion of DBSCAN on MOON, in two phases.

---

## **PROBLEM 3: DBSCAN on real data**
Run the DBSCAN algorithm on the 20NG dataset, and on the FASHION dataset, and the HouseHold dataset (see papers), and evaluate results. 

You need to implement both phases (1) neighborhoods creation, (2) DBSCAN.

Explain why/when it works, and speculate why/when not. 

You need to do trial and error for parameters epsilon and MinPts




*   DBSCAN Revisited, Revisited: Why and How You Should (Still) Use DBSCAN
http://www.ccs.neu.edu/home/vip/teach/DMcourse/2_cluster_EM_mixt/notes_slides/revisitofrevisitDBSCAN.pdf

*   DBSCAN Revisited:Mis-Claim, Un-Fixability, and Approximation
http://www.ccs.neu.edu/home/vip/teach/DMcourse/2_cluster_EM_mixt/notes_slides/sigmod15-dbscan.pdf

EXTRA CREDIT: Using class labels (cheating), try to remove/add points in curate the set for better DBSCAN runs

---

## **PROBLEM 4: Hierarchical Clustering**

Use a library to execute hierarchical clustering on MNIST dataset, evaluate the clusters.